# Fashion MNIST Classification using Convolutional Neural Networks
## Tensorflow-Keras

In [ ]:
## Suppress useless warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from tensorflow.contrib.tensorboard.plugins import projector

%matplotlib inline

## Load the fashion-mnist train & test data

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()
print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)

## Create a 20% validation split from training data

In [ ]:
# create a validation set with 20 percent examples from training set
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size=0.2, random_state=12345)

## Data Normalization

In [ ]:
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Add the color channel axis for Keras operations.
x_train = x_train[..., np.newaxis]
x_validate = x_validate[..., np.newaxis]
x_test = x_test[..., np.newaxis]

## Create Label Dictionary

In [ ]:
label_dict = {
    0: 'T-shirt',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle Boot'
}

## Data Visualization

In [ ]:
fig, ax = plt.subplots(6, 6, figsize = (12, 12))
fig.suptitle('First few images in Fashion MNIST Dataset')
fig.tight_layout(pad = 0.3, rect = [0, 0, 0.9, 0.9])
for x, y in [(i, j) for i in range(6) for j in range(6)]:
    ax[x, y].imshow(x_train[x + y * 6].reshape((28, 28)), cmap = 'gray')
    title = label_dict[int(y_train[x + y * 6])]
    ax[x, y].set_title(title)

## CNN Model Definition

In [ ]:
# Input image dimension:
# WxHxC where W is the Width, H is the Height and C is the number of color channels
img_dim = (x_train[0].shape[0], x_train[0].shape[1], 1)
print("Input image dimension: ", img_dim)

# Define hyper-parameters
batch_size = 128
num_epochs = 10
num_classes = 10

In [ ]:
# Create a Keras Sequential model
model = tf.keras.Sequential()

# Input Convolution layer, f-64, k-2, p-same, a-relu
model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=img_dim))

# Max Pooling Layer, p2
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))

# Dropout of 0.3
model.add(tf.keras.layers.Dropout(0.3))

# Convolution layer, f-32, k-2, p-same, a-relu
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))

# # Max Pooling Layer, p2
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))

# Dropout of 0.3
model.add(tf.keras.layers.Dropout(0.3))

# Flatten the activations of previous layer
model.add(tf.keras.layers.Flatten())

# Fully connected dense layer, fc-256, a-relu
model.add(tf.keras.layers.Dense(256, activation='relu'))

# Dropout of 0.5
model.add(tf.keras.layers.Dropout(0.5))

# Fully connected dense layer, fc-num_classes, a-softmax
model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))

# Take a look at the model summary
model.summary()

## Basic Convolution Operation

<img src="data/convolution_basic.gif">

## Convolution Operation on RGB images using 3D kernel

<img src="data/convolution.gif">

In [ ]:
# Initialize a dictionary to save the training history
history_dict = {}

# Compile the CNN model
# We are using here sparse_categorical_crossentropy loss function and Adam optimizer.
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
    )

history = model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=num_epochs, verbose=1,
    validation_data=(x_validate, y_validate)
    )

history_dict['first_model'] = history

## Plot the Validation accuracy and loss curves

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, figsize=(8, 6))

for history in history_dict:
    val_acc = history_dict[history].history['val_acc']
    val_loss = history_dict[history].history['val_loss']
    ax1.plot(val_acc, label=history)
    ax2.plot(val_loss, label=history)
    
ax1.set_ylabel('validation accuracy')
ax2.set_ylabel('validation loss')
ax2.set_xlabel('epochs')
ax1.legend()
ax2.legend()
plt.show()

## Plot and compare validation and training curves

In [ ]:
accuracy = history_dict[history].history['acc']
val_accuracy = history_dict[history].history['val_acc']
loss = history_dict[history].history['loss']
val_loss = history_dict[history].history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

## Evaluate the model on a test set (not seen by the network)

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

#get the predictions for the test data
predicted_classes = model.predict_classes(x_test)

#get the indices to be plotted
#y_true = test_df.iloc[:, 0]
y_true = y_test
correct = np.nonzero(predicted_classes==y_true)[0]
incorrect = np.nonzero(predicted_classes!=y_true)[0]

target_names = ["Class {}".format(i) for i in range(num_classes)]
print(classification_report(y_true, predicted_classes, target_names=target_names))

## Find embeddings for PCA and tSNE dimensionality reduction

In [ ]:
test_data = np.array(pd.read_csv('data/fashion-mnist_test.csv'), dtype='float32')

embed_count = 900
x_test = test_data[:embed_count, 1:] / 255
y_test = test_data[:embed_count, 0]

logdir = os.getcwd()+'/fmnist_embedding_logdir/' #change this

# setup the write and embedding tensor

summary_writer = tf.summary.FileWriter(logdir)

embedding_var = tf.Variable(x_test, name='fmnist_embedding')

config = projector.ProjectorConfig()
embedding = config.embeddings.add()
embedding.tensor_name = embedding_var.name

embedding.metadata_path = logdir + 'metadata.tsv'
embedding.sprite.image_path = logdir + 'sprite.png'
embedding.sprite.single_image_dim.extend([28, 28])

projector.visualize_embeddings(summary_writer, config)

# run the sesion to create the model check point

with tf.Session() as sesh:
    sesh.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    saver.save(sesh, logdir + 'model.ckpt')
    
    
# create the sprite image and the metadata file

rows = 28
cols = 28

label = ['t_shirt', 'trouser', 'pullover', 'dress', 'coat',
          'sandal', 'shirt', 'sneaker', 'bag', 'ankle_boot']

sprite_dim = int(np.sqrt(x_test.shape[0]))

sprite_image = np.ones((cols * sprite_dim, rows * sprite_dim))

index = 0
labels = []
for i in range(sprite_dim):
    for j in range(sprite_dim):
        
        labels.append(label[int(y_test[index])])
        
        sprite_image[
            i * cols: (i + 1) * cols,
            j * rows: (j + 1) * rows
        ] = x_test[index].reshape(28, 28) * -1 + 1
        
        index += 1
        
with open(embedding.metadata_path, 'w') as meta:
    meta.write('Index\tLabel\n')
    for index, label in enumerate(labels):
        meta.write('{}\t{}\n'.format(index, label))
        
plt.imsave(embedding.sprite.image_path, sprite_image, cmap='gray')
plt.imshow(sprite_image, cmap='gray')
plt.show()

For TensorBoard PCA and tSNE visualization, run this in a new terminal:

**tensorboard --port 8889 --logdir ./fmnist_embedding_logdir/**

After you run this in a terminal, in a new tab go to **yourIP:8889**.

## Assignment

In [ ]:
# Create a Keras Sequential model
model2 = 

# Input Convolution layer, f-128, k-2, p-same, a-relu

# Max Pooling Layer, p2

# Dropout of 0.25

# Input Convolution layer, f-64, k-2, p-same, a-relu

# Max Pooling Layer, p2

# Dropout of 0.25

# Convolution layer, f-32, k-2, p-same, a-relu

# # Max Pooling Layer, p2

# Dropout of 0.3

# Flatten the activations of previous layer

# Fully connected dense layer, fc-256, a-relu

# Dropout of 0.5

# Fully connected dense layer, fc-num_classes, a-softmax

# Take a look at the model summary
model.summary()

In [ ]:
# Initialize a dictionary to save the training history
history_dict = {}

# Compile the CNN model
# We are using here sparse_categorical_crossentropy loss function and Adam optimizer.
model2.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
    )

history = model2.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=num_epochs, verbose=1,
    validation_data=(x_validate, y_validate)
    )

history_dict['second_model'] = history

## Plot the Validation accuracy and loss curves

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, figsize=(8, 6))

for history in history_dict:
    val_acc = history_dict[history].history['val_acc']
    val_loss = history_dict[history].history['val_loss']
    ax1.plot(val_acc, label=history)
    ax2.plot(val_loss, label=history)
    
ax1.set_ylabel('validation accuracy')
ax2.set_ylabel('validation loss')
ax2.set_xlabel('epochs')
ax1.legend()
ax2.legend()
plt.show()

## Plot and compare validation and training curves

In [ ]:
accuracy = history_dict[history].history['acc']
val_accuracy = history_dict[history].history['val_acc']
loss = history_dict[history].history['loss']
val_loss = history_dict[history].history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

## Evaluate the model on a test set (not seen by the network)

In [ ]:
score = model2.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

#get the predictions for the test data
predicted_classes = model2.predict_classes(x_test)

#get the indices to be plotted
y_true = test_df.iloc[:, 0]
correct = np.nonzero(predicted_classes==y_true)[0]
incorrect = np.nonzero(predicted_classes!=y_true)[0]

from sklearn.metrics import classification_report
target_names = ["Class {}".format(i) for i in range(num_classes)]
print(classification_report(y_true, predicted_classes, target_names=target_names))